In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

API KEY

In [2]:
api_key = input("Inserte su API Key de Alpha Vantage: ")

INPUTS

In [ ]:
symbol = input("Inserte el ticker de la empresa/ETF: ").upper()


print("Estas son las opciones: ")
print("1. Balance Sheet")
print("2. Income Statement")
print("3. Cash Flow")
print("4. Historial de dividendos")
print("5. Resumen de la empresa")
print("6. Perfil de ETFs y tenencias")
print("7. EPS")
print("8. Precios históricos")

while True:
  try:
    opcion = int(input("Inserte la opción que desea: "))
    if opcion < 1 or opcion > 8:
      print("Inserte una opción válida")
    else:
      break
  except ValueError:
    print("Inserte una opción válida")

while True:
  if opcion == 1:
    sheet = "BALANCE_SHEET"
    break
  elif opcion == 2:
    sheet = "INCOME_STATEMENT"
    break
  elif opcion == 3:
    sheet = "CASH_FLOW"
    break
  elif opcion == 4:
    sheet = "DIVIDENDS"
    break
  elif opcion == 5:
    sheet = "OVERVIEW"
    break
  elif opcion == 6:
    sheet = "ETF_PROFILE"
    break
  elif opcion == 8:
    sheet = "TIME_SERIES_DAILY"
    break
  elif opcion == 7:
    sheet = "EARNINGS"
    break

In [4]:
if opcion >= 1 and opcion <= 7:
  url = f'https://www.alphavantage.co/query?function={sheet}&symbol={symbol}&apikey={api_key}'

  r = requests.get(url)
  data = r.json()
else:
  url = f'https://www.alphavantage.co/query?function={sheet}&symbol={symbol}&outputsize=full&apikey={api_key}'

  r = requests.get(url)
  data = r.json()

DATAFRAME A EXCEL

In [5]:
if opcion >= 1 and opcion <= 6:
  if 'annualReports' in data:
      reports = data['annualReports']
      df = pd.DataFrame(reports)

      df.to_excel(f'{symbol}_{sheet}.xlsx')
  else:
      print("La clave 'annualReports' no se encontró en la respuesta JSON")
elif opcion == 7:
  if 'annualEarnings' in data:
    earnings = data['annualEarnings']
    df = pd.DataFrame(earnings)

    df.to_excel(f'{symbol}_{sheet}.xlsx')
elif opcion == 8:
  if 'Time Series (Daily)' in data:
    time_series = data['Time Series (Daily)']
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df[['4. close']].rename(columns={'4. close': 'Close'})

    # Convertir la columna 'Close' a tipo float
    df['Close'] = df['Close'].astype(float)

    # Ordenar el DataFrame por fecha
    df = df.sort_index()

    # Guardar el DataFrame como un archivo Excel
    df.to_excel(f'{symbol}_precios.xlsx')

    print(df)
  else:
    print("No se encontraron datos de precios en la respuesta JSON")

GRAFICO PRECIO

In [ ]:
if opcion == 8:
  plt.figure(figsize=(10, 6))
  plt.plot(df.index, df['Close'], label='Precio de Cierre')
  plt.xlabel('Fecha')
  plt.ylabel('Precio de Cierre (USD)')
  plt.title(f'Precios diarios de {symbol}')
  plt.legend()
  plt.grid(True)

  # Usar MaxNLocator para ajustar los ticks del eje Y
  plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))

  # Establecer el rango del eje Y desde 0 hasta el valor máximo de cierre
  plt.ylim(0, df['Close'].max() + (df['Close'].max() * 0.1))  # Añadimos un 10% extra al valor máximo para mejor visualización

  # Mostrar el gráfico
  plt.show()